In [3]:
import numpy as np
import pandas as pd
import pyspark

In [4]:
from sklearn.model_selection import cross_validate
# from sklearn.metrics import make_scorer

In [5]:
from hyperopt import fmin, hp, tpe
from hyperopt import SparkTrials, STATUS_OK

In [6]:
from functools import partial

In [7]:
import lightgbm as lgbm

In [107]:
def objective(clf, params):
    cv = cross_validate(clf.set_params(**params), X_train, y_train, cv=3, scoring='roc_auc',
                        verbose=1, n_jobs=-1,
                        return_train_score=True, return_estimator=False)
    
    train_gini = np.mean((2*cv['train_score'] - 1))
    test_gini = np.mean((2*cv['test_score'] - 1))
    
    x = np.abs(train_gini - cv_gini) 
    delta, std = 0.03, 0.002
    f_val = -np.piecewise(x, [x < delta, x >= delta], [cv_gini, cv_gini*(np.exp(-0.5*((x-delta)**2)/std))])
    return f_val

In [108]:
params = {
        'num_leaves': 30,
        'colsample_bytree': 0.15
}

space = {
    'learning_rate' : hp.loguniform('learning_rate', np.log(0.001), np.log(0.2)),
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0)
}

In [ ]:
fmin_objective = partial(objective, clf = lgbm.LGBMModel() ,params = params)

In [112]:
fmin_objective = partial(objective, clf = lgbm.LGBMModel() ,params = params)

# spark_trials = SparkTrials()
h_star = fmin(
            fn = fmin_objective,
            algo = tpe.suggest,
            space = space,
            max_evals=10
#             trials=spark_trials,
            )

  0%|          | 0/9223372036854775807 [00:00<?, ?trial/s, best loss=?]

job exception: objective() got multiple values for argument 'clf'



  0%|          | 0/9223372036854775807 [00:00<?, ?trial/s, best loss=?]


TypeError: objective() got multiple values for argument 'clf'